In [3]:
import pandas as pd
import numpy as np
import numpy as numpy
from numpy import array
from numpy import argmax
np.random.seed(7) # for keras reproducibility, set before import as shown in keras examples. https://github.com/keras-team/keras/issues/2743
from sklearn import datasets, metrics, preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer,StandardScaler, LabelEncoder, OneHotEncoder 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix, mean_squared_error, matthews_corrcoef
from sklearn.datasets import make_classification, load_digits
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

import dask_ml.model_selection as dcv

import matplotlib
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import regularizers
from keras.regularizers import l1
reg = l1(0.001)

from keras.callbacks import TensorBoard
from time import time
from keras import optimizers

import eli5
from eli5.sklearn import PermutationImportance
from itertools import cycle
from scipy import interp

In [4]:
dataset= pd.read_csv('BP_training1.csv')
data = dataset.drop(["gene"],1)
print(data.shape)

(303, 40)


In [5]:
df = data.iloc[:,0:39]
imputer = SimpleImputer(missing_values = np.nan, strategy = "mean")
imputer = imputer.fit(df)
df = imputer.transform(df)
X = MinMaxScaler().fit_transform(df)
le = preprocessing.LabelEncoder()

Y = data["category"]

### Investigating accuracies of different builds starting from iMEGES build:

In [ ]:
seed = 7

#iMEGES model (adapted for multiclass output of 4)
def baseline_model(learn_rate=0.01):
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(39,)))
    model.add(Dense(50, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(40, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    #model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    #model.add(Dense(20, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(10, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(4, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

keras = KerasClassifier(build_fn=baseline_model,batch_size=32, epochs=150, verbose=0)


learn_rate = [0.001, 0.01]
kerasparams = dict(learn_rate=learn_rate)


inner_cv = KFold(n_splits=5, shuffle=True, random_state=seed)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=seed)

models = []

models.append(('Keras', GridSearchCV(estimator=keras, param_grid=kerasparams, cv=inner_cv,iid=False, n_jobs=-1)))

results = []
names = []
scoring = 'accuracy'



X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


In [5]:
for name, model in models:
    nested_cv_results = model_selection.cross_val_score(model, X, Y, cv=outer_cv, scoring=scoring)
    results.append(nested_cv_results)
    names.append(name)
    msg = "Nested CV Accuracy %s: %f (+/- %f )" % (name, nested_cv_results.mean()*100, nested_cv_results.std()*100)
    print(msg)
    model.fit(X_train, Y_train)
    print('Test set accuracy: {:.2f}'.format(model.score(X_test, Y_test)*100),  '%')
    print("Best Parameters: \n{}\n".format(model.best_params_))
    print("Best CV Score: \n{}\n".format(model.best_score_)) #average of all cv folds for a single combination of the parameters you specify 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


D:\Anaconda3\envs\tensorflow\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Nested CV Accuracy Keras: 70.316940 (+/- 6.218800 )
Test set accuracy: 60.66 %
Best Parameters: 
{'learn_rate': 0.001}

Best CV Score: 
0.694217689994241



In [6]:
#reducing number of layers and adding l1 regularisation:
#using softmax which is more commonly used for multiclass problems (https://medium.com/aidevnepal/for-sigmoid-funcion-f7a5da78fec2)

def baseline_model(learn_rate=0.01):
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(39,)))
    model.add(Dense(50, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(40, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(30, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(20, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(20, activation='relu',activity_regularizer=l1(0.0001)))
    model.add(Dense(10, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

keras = KerasClassifier(build_fn=baseline_model,batch_size=32, epochs=150, verbose=0)

for name, model in models:
    nested_cv_results = model_selection.cross_val_score(model, X, Y, cv=outer_cv, scoring=scoring)
    results.append(nested_cv_results)
    names.append(name)
    msg = "Nested CV Accuracy %s: %f (+/- %f )" % (name, nested_cv_results.mean()*100, nested_cv_results.std()*100)
    print(msg)
    model.fit(X_train, Y_train)
    print('Test set accuracy: {:.2f}'.format(model.score(X_test, Y_test)*100),  '%')
    print("Best Parameters: \n{}\n".format(model.best_params_))
    print("Best CV Score: \n{}\n".format(model.best_score_)) #average of all cv folds for a single combination of the parameters you specify 


Nested CV Accuracy Keras: 64.677596 (+/- 10.066790 )
Test set accuracy: 62.30 %
Best Parameters: 
{'learn_rate': 0.001}

Best CV Score: 
0.6857142881471283



In [7]:
#reducing number of layers and adding l1 regularisation:
#using softmax which is more commonly used for multiclass problems (https://medium.com/aidevnepal/for-sigmoid-funcion-f7a5da78fec2)
#increasing the dropout

def baseline_model(learn_rate=0.01):
    model = Sequential()
    model.add(Dropout(0.5, input_shape=(39,)))
    model.add(Dense(50, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(40, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(30, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(20, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(20, activation='relu',activity_regularizer=l1(0.0001)))
    model.add(Dense(10, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

keras = KerasClassifier(build_fn=baseline_model,batch_size=32, epochs=150, verbose=0)

for name, model in models:
    nested_cv_results = model_selection.cross_val_score(model, X, Y, cv=outer_cv, scoring=scoring)
    results.append(nested_cv_results)
    names.append(name)
    msg = "Nested CV Accuracy %s: %f (+/- %f )" % (name, nested_cv_results.mean()*100, nested_cv_results.std()*100)
    print(msg)
    model.fit(X_train, Y_train)
    print('Test set accuracy: {:.2f}'.format(model.score(X_test, Y_test)*100),  '%')
    print("Best Parameters: \n{}\n".format(model.best_params_))
    print("Best CV Score: \n{}\n".format(model.best_score_)) #average of all cv folds for a single combination of the parameters you specify 


Nested CV Accuracy Keras: 67.672131 (+/- 3.228135 )
Test set accuracy: 62.30 %
Best Parameters: 
{'learn_rate': 0.01}

Best CV Score: 
0.6694727902104254



In [8]:
#reducing number of layers and adding l1 regularisation:
#using softmax which is more commonly used for multiclass problems (https://medium.com/aidevnepal/for-sigmoid-funcion-f7a5da78fec2)
#changing the number of neurons

def baseline_model(learn_rate=0.01):
    model = Sequential()
    model.add(Dense(100, input_dim=X.shape[1], activation='relu', activity_regularizer=l1(0.0001)))
    model.add(Dropout(0.2))
    model.add(Dense(75, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(50, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(25, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

keras = KerasClassifier(build_fn=baseline_model,batch_size=32, epochs=150, verbose=0)

for name, model in models:
    nested_cv_results = model_selection.cross_val_score(model, X, Y, cv=outer_cv, scoring=scoring)
    results.append(nested_cv_results)
    names.append(name)
    msg = "Nested CV Accuracy %s: %f (+/- %f )" % (name, nested_cv_results.mean()*100, nested_cv_results.std()*100)
    print(msg)
    model.fit(X_train, Y_train)
    print('Test set accuracy: {:.2f}'.format(model.score(X_test, Y_test)*100),  '%')
    print("Best Parameters: \n{}\n".format(model.best_params_))
    print("Best CV Score: \n{}\n".format(model.best_score_)) #average of all cv folds for a single combination of the parameters you specify 


Nested CV Accuracy Keras: 70.322404 (+/- 5.952282 )
Test set accuracy: 60.66 %
Best Parameters: 
{'learn_rate': 0.001}

Best CV Score: 
0.7105442198766332



In [9]:
#increasing the dropout:

def baseline_model(learn_rate=0.01):
    model = Sequential()
    model.add(Dense(100, input_dim=X.shape[1], activation='relu', activity_regularizer=l1(0.0001)))
    model.add(Dropout(0.5))
    model.add(Dense(75, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(50, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(25, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

keras = KerasClassifier(build_fn=baseline_model,batch_size=32, epochs=150, verbose=0)

for name, model in models:
    nested_cv_results = model_selection.cross_val_score(model, X, Y, cv=outer_cv, scoring=scoring)
    results.append(nested_cv_results)
    names.append(name)
    msg = "Nested CV Accuracy %s: %f (+/- %f )" % (name, nested_cv_results.mean()*100, nested_cv_results.std()*100)
    print(msg)
    model.fit(X_train, Y_train)
    print('Test set accuracy: {:.2f}'.format(model.score(X_test, Y_test)*100),  '%')
    print("Best Parameters: \n{}\n".format(model.best_params_))
    print("Best CV Score: \n{}\n".format(model.best_score_)) #average of all cv folds for a single combination of the parameters you specify 


Nested CV Accuracy Keras: 69.349727 (+/- 7.599842 )
Test set accuracy: 68.85 %
Best Parameters: 
{'learn_rate': 0.001}

Best CV Score: 
0.6898809550570792



In [11]:
#reducing number of layers

def baseline_model(learn_rate=0.01):
    model = Sequential()
    model.add(Dense(100, input_dim=X.shape[1], activation='relu', activity_regularizer=l1(0.0001)))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(25, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

keras = KerasClassifier(build_fn=baseline_model,batch_size=32, epochs=150, verbose=0)

for name, model in models:
    nested_cv_results = model_selection.cross_val_score(model, X, Y, cv=outer_cv, scoring=scoring)
    results.append(nested_cv_results)
    names.append(name)
    msg = "Nested CV Accuracy %s: %f (+/- %f )" % (name, nested_cv_results.mean()*100, nested_cv_results.std()*100)
    print(msg)
    model.fit(X_train, Y_train)
    print('Test set accuracy: {:.2f}'.format(model.score(X_test, Y_test)*100),  '%')
    print("Best Parameters: \n{}\n".format(model.best_params_))
    print("Best CV Score: \n{}\n".format(model.best_score_)) #average of all cv folds for a single combination of the parameters you specify 


Nested CV Accuracy Keras: 67.005464 (+/- 8.395061 )
Test set accuracy: 65.57 %
Best Parameters: 
{'learn_rate': 0.001}

Best CV Score: 
0.6818877582647362



In [10]:
#increasing the dropout: 

def baseline_model(learn_rate=0.01):
    model = Sequential()
    model.add(Dense(100, input_dim=X.shape[1], activation='relu', activity_regularizer=l1(0.0001)))
    model.add(Dropout(0.5))
    model.add(Dense(75, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(50, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(25, activation='relu',activity_regularizer=l1(0.0001))) 
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

keras = KerasClassifier(build_fn=baseline_model,batch_size=32, epochs=150, verbose=0)

for name, model in models:
    nested_cv_results = model_selection.cross_val_score(model, X, Y, cv=outer_cv, scoring=scoring)
    results.append(nested_cv_results)
    names.append(name)
    msg = "Nested CV Accuracy %s: %f (+/- %f )" % (name, nested_cv_results.mean()*100, nested_cv_results.std()*100)
    print(msg)
    model.fit(X_train, Y_train)
    print('Test set accuracy: {:.2f}'.format(model.score(X_test, Y_test)*100),  '%')
    print("Best Parameters: \n{}\n".format(model.best_params_))
    print("Best CV Score: \n{}\n".format(model.best_score_)) #average of all cv folds for a single combination of the parameters you specify 


Nested CV Accuracy Keras: 71.961749 (+/- 5.513019 )
Test set accuracy: 60.66 %
Best Parameters: 
{'learn_rate': 0.001}

Best CV Score: 
0.6776360575844642



71.96% is the highest accuracy with nested CV minismsing the risk of this being due to overfitting

## Deciding Epoch number with optimised model:

- Runnining optimised model within tensorboard (results written to 'Keras_BP' folder and loaded via anaconda prompt)

In [14]:
model = Sequential()
model.add(Dense(100, input_dim=X.shape[1], activation='relu', activity_regularizer=l1(0.0001)))
model.add(Dropout(0.5))
model.add(Dense(75, activation='relu',activity_regularizer=l1(0.0001))) 
model.add(Dense(50, activation='relu',activity_regularizer=l1(0.0001))) 
model.add(Dense(25, activation='relu',activity_regularizer=l1(0.0001))) 
model.add(Dense(4, activation='softmax'))
keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
model.compile(loss='sparse_categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="Keras_BP/{}".format(time()))

model.fit(X_train, Y_train,  validation_data=(X_test, Y_test), verbose=1,  batch_size=32, epochs=500,callbacks=[tensorboard])

Train on 242 samples, validate on 61 samples
Epoch 1/500
242/242 [==============================] - 2s 7ms/step - loss: 1.4434 - acc: 0.3388 - val_loss: 1.3795 - val_acc: 0.2787
Epoch 2/500
242/242 [==============================] - 0s 151us/step - loss: 1.3534 - acc: 0.4008 - val_loss: 1.2866 - val_acc: 0.5410
Epoch 3/500
242/242 [==============================] - 0s 173us/step - loss: 1.2523 - acc: 0.5661 - val_loss: 1.1345 - val_acc: 0.6721
Epoch 4/500
242/242 [==============================] - 0s 163us/step - loss: 1.1192 - acc: 0.6364 - val_loss: 0.9471 - val_acc: 0.6721
Epoch 5/500
242/242 [==============================] - 0s 155us/step - loss: 0.9785 - acc: 0.6570 - val_loss: 0.8533 - val_acc: 0.6721
Epoch 6/500
242/242 [==============================] - 0s 180us/step - loss: 0.9402 - acc: 0.6529 - val_loss: 0.8567 - val_acc: 0.6557
Epoch 7/500
242/242 [==============================] - 0s 205us/step - loss: 0.9007 - acc: 0.6653 - val_loss: 0.8643 - val_acc: 0.6557
Epoch 8/500


Epochs within tensorboard show plateau as  ~100 epochs

In [32]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://imgur.com/Nwbos8a.png")

#Y-axis is accuracy
#X-axis is epochs